In [205]:
import pandas as pd
import fuzzymatcher
import os
import re
pd.options.mode.chained_assignment = None

In [206]:
path = "../../res"
mapping_path = "../../res/matching"

In [224]:
db_table_path = "../../Database/table"
df_phones = pd.read_csv(os.path.join(db_table_path, "phones.csv"), sep =";")
df_sources = pd.read_csv(os.path.join(db_table_path, "sources.csv"), sep =";")
df_sources.astype(str)
df_phones.astype(str)
df_phones['id'] = df_phones['id'].astype('int64')
df_sources['phone_id'] = df_sources['phone_id'].astype('int64')

warehouse = {}
warehouse['phones'] = df_phones
warehouse['sources'] = df_sources

int_field = ["ram", "rom", "battery", "monitor_frequency", "phone_id", "int", 'price']

In [208]:
def getSource(souce_name, sep=";"):
    df = pd.read_csv(os.path.join(path, souce_name+".csv"), sep = sep)
    df['id'] = df.index
    df.astype(str)
    return df
def getMapping(source_name, sep=";"):
    df = pd.read_csv(os.path.join(mapping_path, source_name+".txt"), sep = sep)
    mapping = {}
    for i in range(df.shape[0]):
        mapping[df['schema'][i]] = df[source_name][i]
    return mapping

In [209]:
cellphones = getSource("cellphones")
mapping_cellphones = getMapping("cellphones")

hhm = getSource("hhm")
hhm['source'] = 'Hoàng Hà mobile'
mapping_hhm = getMapping("hhm")

nk = getSource("nk")
nk['source'] = 'Nguyễn Kim mobile'
mapping_nk = getMapping("nk")

tgdd = getSource("tgdd")
tgdd['source'] = 'Thế giới di động'
mapping_tgdd = getMapping('tgdd')


In [210]:
def load_data(source, target, mapping):
    t = target.copy()
    for key in mapping:
        for table in t:
            t[table] = t[table].copy()
            df_table = t[table]
            if key in df_table.columns:
                target_table = df_table
        target_table[key] = source[mapping[key]]

    t['sources']['phone_id'] = source['id']
    t['phones']['id'] = source['id']
    return t


In [211]:
def load_match_data(source, target, id_mapping, mapping):
    t = {}
    phones = target['phones'].copy()
    sources = target['sources'].copy()
    df_not_matched = pd.DataFrame()
    df_matched = pd.DataFrame()

    df_not_matched= source[~source['id'].isin(id_mapping.keys())]
    df_matched = source[source['id'].isin(id_mapping.keys())]

    for index, row in df_matched.iterrows():
        data_row_left = {}
        for field in source.columns:
            data_row_left[field] = row[field]
        id_left = data_row_left['id']
        row_right = phones.loc[phones['id'] == id_mapping[id_left]]
        for index, row in row_right.iterrows():
            for field in phones.columns:
                if field in mapping:
                    if str(phones.at[index, field]) == 'nan':
                        phones.at[index, field] = data_row_left[mapping[field]]

        data_source = {}
        for field in sources.columns:
            if field in mapping:
                data_source[field] = (data_row_left[mapping[field]])
            elif field =='phone_id':
                data_source[field] = (id_mapping[id_left])
            else :
                data_source[field] = ("")

        data_source = pd.DataFrame([data_source])
        sources = pd.concat([sources,data_source] , axis=0, ignore_index=True)


    #if object is not exist in warehouse
    for index, row in df_not_matched.iterrows():
        data_row = {}
        for field in source.columns:
            data_row[field] = row[field]
        data_row['id'] = phones.shape[0]
        data_phone = {}
        for field in phones.columns:
            if field in mapping:
                data_phone[field] = (data_row[mapping[field]])
            elif field == 'id':
                data_phone['id'] = (data_row['id'])
            else :
                data_phone[field] = ("")

        data_source = {}
        for field in sources.columns:
            if field in mapping:
                data_source[field] = (data_row[mapping[field]])
            elif field =='phone_id':
                data_source[field] = (data_row['id'])
            else :
                data_source[field] = ("")

        data_phone = pd.DataFrame([data_phone])
        data_source = pd.DataFrame([data_source])
        phones = pd.concat([phones, data_phone] , axis=0, ignore_index=True)
        sources = pd.concat([sources,data_source] , axis=0, ignore_index=True)

    t['phones'] = phones
    t['sources'] = sources
    return t


In [212]:
def nomalize_data(df, mapping):
    df = df.copy()
    for key in mapping:
        if key in int_field and df[mapping[key]].dtype == 'object':
            for i in range(df.shape[0]):
                a = ''.join(filter(str.isdigit, str(df[mapping[key]][i])))
                df[mapping[key]][i] = a
    return df

In [213]:
def data_matching(left_df, left_col, right_df, right_col):
    df = fuzzymatcher.fuzzy_left_join(left_df, right_df, left_col, right_col, left_id_col = "id", right_id_col = "id")
    return df

In [214]:
def get_data_match(df_source, target, mapping, min_accept=0.5):
    if (target['phones'].shape[0] == 0) :
        return pd.DataFrame()
    target_cols = ["name", "ram", "rom"]
    source_cols = [mapping[x] for x in target_cols]
    data_match = data_matching(df_source , source_cols, target['phones'], target_cols)
    data_match = data_match.sort_values(by=['best_match_score'], ascending=False)
    data_match = data_match[data_match['best_match_score'] > min_accept]
    return data_match

    

In [215]:
def show_data_match(data_match, mapping):
   target_cols = ["name", "ram", "rom"]
   show_col = ['best_match_score', '__id_left', '__id_right']
   for col in target_cols:
      if (mapping[col] == col):
         show_col.append(col+'_left')
         show_col.append(col+'_right')
      else:
         show_col.append(col)
         show_col.append(mapping[col])

   data = data_match[data_match.columns.intersection(show_col)]
   return data

In [216]:
def data_maping(df_source, target, mapping, min_accept=0.5):
    df_source = nomalize_data(df_source,mapping)
    data = get_data_match(df_source, target, mapping, min_accept)

    if (data.empty):
        warehouse = load_data(df_source, target,mapping)
        return warehouse

    id_left = data['__id_left'].to_list()
    id_right = data['__id_right'].to_list()
    
    id_mapping = {}
    for i in range(len(id_left)):
        id_mapping[id_left[i]] = id_right[i]

    t = load_match_data(df_source, target, id_mapping, mapping)
    return t

# MAPPING DATA

In [225]:
warehouse = data_maping(cellphones, warehouse, mapping_cellphones)


In [228]:
warehouse = data_maping(hhm, warehouse, mapping_hhm)

In [231]:
warehouse = data_maping(nk, warehouse, mapping_nk, 0.1)

In [234]:
warehouse = data_maping(tgdd, warehouse, mapping_tgdd, 0.3)

# Output

In [241]:
warehouse['sources']['phone_id'] = warehouse['sources']['phone_id'].astype('int64')

In [240]:
warehouse['phones'].to_csv(os.path.join(db_table_path, "phones.csv"), sep =";",index=False)
warehouse['sources'].to_csv(os.path.join(db_table_path, "sources.csv"), sep =";", index=False)